# This notebook reads an excel file with ISSNand retrives journal information from Scimago. ISSns should not contains "-".

In [48]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
df = pd.read_excel('deduplicated.xlsx')

In [49]:
len(df[['ISSN']].drop_duplicates().values)

1780

In [50]:
dfu = df['ISSN'].value_counts().reset_index()
dfu.columns = ['ISSN', '# papers']

In [51]:
def finding_page(iss):
    while True:
        try:
            search_base = 'https://www.scimagojr.com/journalsearch.php?q='
            url = search_base+iss
            res = requests.get(url).text
            soup = BeautifulSoup(res)
            s = 'Sorry, no results were found.'
            if s in soup.text:
                output.append(row)
                print('No result!')
                print(iss, 't2')
                return None
            res = soup.findAll('div',{'class':"search_results"})
            journal_relative_url = [x['href'] for x in res[0].findAll('a')][0]
            break
        except:
            time.sleep(60)
    return journal_relative_url

def get_page(journal_relative_url):
    journal_full_url = 'https://www.scimagojr.com/'+journal_relative_url
    journal_page = requests.get(journal_full_url).text
    soup = BeautifulSoup(journal_page)
    return soup

In [ ]:
output = []
subjects = []
from tqdm.notebook import tqdm
for i, row in enumerate(dfu.values):
    print(i)
    iss, journal = row
    print(iss)
    row = ['']*6
    row[3] = journal
    row[-1] = iss
    if type(iss) == float:
        output.append(row)
        print(iss, 't1')
        continue
    
    
    while True:
        journal_relative_url = finding_page(iss)
        break
    print(journal_relative_url)
    if not journal_relative_url:
        continue
    journal_full_url = 'https://www.scimagojr.com/'+journal_relative_url
    
    while True:
         try:
            soup = get_page(journal_relative_url)
            if '500 Internal Server Error' in soup.text:
                time.sleep(60)
                continue
            break
         except:
             time.sleep(60)
    
    issns = soup.findAll('div', {'class': 'journalgrid'})[0].findAll('div')[5].findAll('p')[0].text.split(', ')
    issns = [x.strip('X').strip('x') for x in issns]
    if not iss.strip('x').strip('X') in issns:
        row = ['']*6
        row[3] = journal
        row[-1] = iss
        output.append(row)
        print(iss, 't4')
        continue
    #print(journal, issn)
    s = "Sorry, no results were found."
    tabels = soup.findAll('div', {'class': 'cellslide'})
    q = []
    q.append(journal)
    sjr = [x.text for x in tabels[3].findAll('tr')[-1].findAll('td')][1]
    q.append(sjr)
    country = soup.findAll('div', {'class': 'journalgrid'})[0].findAll('a')[0].text
    q.append(country)
    h = soup.find('p', {'class': 'hindexnumber'}).text
    q.append(h)
    q.append(iss)
    if len(soup.findAll(attrs={'alt': 'open access'}))>0:
        q.append(1)
    else:
        q.append(0)
    output.append(q)
    try:
        qs = [[y.text for y in x.findAll('td')]+[iss] for x in tabels[1].findAll('tr')[1:] if x.findAll('td')[1].text=='2022']
    except:
        qs = [[x.text, '2022', '', iss] for x in soup.findAll('div', {'class': 'journalgrid'})[0].findAll('div')[1].findAll('a')  if '00' not in x['href']]
    subjects.extend(qs)
    time.sleep(2)

0
20080387
journalsearch.php?q=19700187808&tip=sid&clean=0
1
09697330
journalsearch.php?q=28836&tip=sid&clean=0
2
00309982
journalsearch.php?q=16479&tip=sid&clean=0
3
14726939
journalsearch.php?q=28100&tip=sid&clean=0
4
10203397
journalsearch.php?q=20214&tip=sid&clean=0
5
00411345
journalsearch.php?q=19279&tip=sid&clean=0
6
22516085
journalsearch.php?q=21484&tip=sid&clean=0
7
1682024X
journalsearch.php?q=16469&tip=sid&clean=0
8
1022386X
journalsearch.php?q=29808&tip=sid&clean=0
9
03795284
journalsearch.php?q=18826&tip=sid&clean=0
10
22779531
journalsearch.php?q=21100904390&tip=sid&clean=0
11
19967195
journalsearch.php?q=21100215707&tip=sid&clean=0
12
10292977
journalsearch.php?q=27539&tip=sid&clean=0
13
14726920
journalsearch.php?q=28099&tip=sid&clean=0
14
14726955
journalsearch.php?q=27155&tip=sid&clean=0
15
13192442
journalsearch.php?q=5000154608&tip=sid&clean=0
16
03066800
journalsearch.php?q=15962&tip=sid&clean=0
17
0142159X
journalsearch.php?q=18357&tip=sid&clean=0
18
19326203
jou

In [ ]:
pd.DataFrame([x for x in output], columns = ['Count', 'SJR', 'Country', 'Hindex', 'ISSN', 'Open Access'])#.to_excel('journals measures.xlsx')

In [ ]:
temp = pd.DataFrame(subjects, columns = ['Subject', 'Year', 'Q', 'ISSN'])
del temp['Year']
temp#.to_excel('journals subjects.xlsx')